In [28]:
import pandas as pd
import numpy as np

In [29]:
path = "../../raw_data/raw_twitter_60min_sentiment_df_v1.csv"
data = pd.read_csv(path)
df = data.copy()
df.dtypes

hourly_truncated_time     object
transformed_ratio        float64
ratio                    float64
dtype: object

In [30]:
df.head()

,hourly_truncated_time,transformed_ratio,ratio
0,2022-01-11 18:00,8.472926,9.584746
1,2022-01-11 19:00,5.103936,5.045161
2,2022-01-11 20:00,3.106815,3.023474
3,2022-01-11 21:00,4.194768,4.260274
4,2022-01-11 22:00,5.409466,5.119205


In [31]:
def df_formating(df, columns):
    # Step 1: Rename columns
    formated_df = df.rename(columns={
        columns[0]: 'date',
        columns[1]: 'transformed_ratio',
        columns[2]: 'ratio'
    })

    # Step 2: Drop all other columns
    columns_to_keep = ['date', 'transformed_ratio', 'ratio']
    formated_df = formated_df[columns_to_keep]

    # Step 3: Set columns to float32
    formated_df = formated_df.astype({'transformed_ratio': 'float32', 'ratio': 'float32'})

    # Step 4: Set 'date' column to datetime type
    formated_df['date'] = pd.to_datetime(formated_df['date'], format='mixed')

    # Step 5: Set 'date' column as the index
    formated_df.set_index('date', inplace=True)

    return formated_df

In [32]:
columns = ['hourly_truncated_time', 'transformed_ratio', 'ratio']
formated_df = df_formating(df, columns)
formated_df.head()

,transformed_ratio,ratio
date,,
2022-01-11 18:00:00,8.472926,9.584745
2022-01-11 19:00:00,5.103936,5.045161
2022-01-11 20:00:00,3.106815,3.023474
2022-01-11 21:00:00,4.194768,4.260274
2022-01-11 22:00:00,5.409466,5.119205


In [33]:
formated_df.dtypes

transformed_ratio    float32
ratio                float32
dtype: object

In [34]:
formated_df.to_csv('../../raw_data/pro_twitter_60min_sentiment_df_v2.csv')

In [12]:
def invert_rows_order(df):
    """
    Invert the order of rows in a DataFrame.

    Parameters:
    - df: Input DataFrame.

    Returns:
    - inverted_df: DataFrame with rows in reversed order.
    """
    inverted_df = df.iloc[::-1]
    return inverted_df

In [10]:
inverted_formated_df = invert_rows_order(formated_df)
inverted_formated_df.head()

,open,high,low,adj_close,volume
date,,,,,
2015-10-08 13:40:00,0.000000,242.500000,0.000000,242.500000,12.125000
2015-10-08 13:41:00,242.500000,242.949997,242.500000,242.949997,0.242950
2015-10-08 13:42:00,242.949997,242.949997,242.949997,242.949997,0.000000
2015-10-08 13:43:00,242.949997,242.960007,242.949997,242.960007,2.429600
2015-10-08 13:44:00,242.960007,242.960007,242.960007,242.960007,8.137003


In [13]:
def convert_to_interval(input_file, output_file, interval_minutes):
    """
    Convert a CSV file with minute-level data to a new CSV file with a specified time interval.

    Parameters:
    - input_file (str): Path to the input CSV file.
    - output_file (str): Path to the output CSV file.
    - interval_minutes (int): Time interval for each row (e.g., 5 or 10 minutes).
    """
    # Read the input CSV file into a DataFrame
    df = pd.read_csv(input_file, index_col=0, parse_dates=True)

    df.index = pd.to_datetime(df.index)

    # Resample the DataFrame based on the specified time interval
    resampled_df = df.resample(f'{interval_minutes}T').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'adj_close': 'last',
        'volume': 'sum'
    })

    # Drop rows with missing values (due to resampling)
    resampled_df = resampled_df.dropna()

    resampled_df = resampled_df.astype({'open': 'float32', 'high': 'float32', 'low': 'float32', 'adj_close': 'float32', 'volume': 'float32'})

    # Write the resampled DataFrame to a new CSV file
    resampled_df.to_csv(output_file)

    return resampled_df

In [14]:
input_csv_path = '../../raw_data/pro_btc_1min_price_df_v3.csv'
output_csv_path = '../../raw_data/pro_btc_60min_price_df_v3.csv'
df_diff = convert_to_interval(input_csv_path, output_csv_path, interval_minutes=60)

KeyError: "Column(s) ['adj_close', 'high', 'low', 'open', 'volume'] do not exist"

In [48]:
df_diff.dtypes

open         float32
high         float32
low          float32
adj_close    float32
volume       float32
dtype: object